In [ ]:
import spacy
from symspellpy import SymSpell

# Initialize SymSpell
sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)

# Load the standard dictionary
sym_spell.load_dictionary("./frequency_dictionary_en_82_765.txt", term_index=0, count_index=1)
nlp = spacy.load("en_core_web_md")

def correct_spelling(text):
    suggestions = sym_spell.lookup_compound(text, max_edit_distance=3)
    return suggestions[0].term if suggestions else text

def is_word_in_dictionary(word):
    return word in sym_spell.words

def correct_proper_noun(proper_nouns):
    proper_noun = correct_spelling(proper_nouns)
    if is_word_in_dictionary(proper_noun):
        return proper_noun
    sym_spell.create_dictionary_entry(proper_nouns, 1)
    return proper_nouns

# 텍스트를 NER 형태로 변환하는 함수
def convert_to_ner_format(text):
    doc = nlp(text)
    ner_formatted_text = ""
    last_end = 0

    for ent in doc.ents:
        # 엔터티 전까지의 일반 텍스트 추가
        ner_formatted_text += correct_spelling(text[last_end:ent.start_char]) + ' '
        # 엔터티와 그 태그 추가
        ner_formatted_text += correct_proper_noun(ent.text) + ' '
        last_end = ent.end_char

    # 마지막 엔터티 이후의 일반 텍스트 추가
    ner_formatted_text += correct_spelling(text[last_end:])
    if len(ner_formatted_text)==0:
        ner_formatted_text = text
    elif ner_formatted_text[-1]==' ':
        ner_formatted_text = ner_formatted_text[:-1] + '.\n'
    return ner_formatted_text

# 오타를 수정하는 함수
def correct_spelling(text):
    suggestions = sym_spell.lookup_compound(text, max_edit_distance=3)
    return suggestions[0].term if suggestions else text